In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [5]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [6]:
mlp = MLPClassifier()
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000]
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [7]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [ ]:
bertweet_dynahate_train_embeddings = reshape_embeddings(bertweet_dynahate_train_embeddings)
bertweet_dynahate_dev_embeddings = reshape_embeddings(bertweet_dynahate_dev_embeddings)
bertweet_dynahate_test_embeddings = reshape_embeddings(bertweet_dynahate_test_embeddings)

bertweet_latenthatred_train_embeddings = reshape_embeddings(bertweet_latenthatred_train_embeddings)
bertweet_latenthatred_dev_embeddings = reshape_embeddings(bertweet_latenthatred_dev_embeddings)
bertweet_latenthatred_test_embeddings = reshape_embeddings(bertweet_latenthatred_test_embeddings)

bertweet_olid_train_embeddings = reshape_embeddings(bertweet_olid_train_embeddings)
bertweet_olid_dev_embeddings = reshape_embeddings(bertweet_olid_dev_embeddings)
bertweet_olid_test_embeddings = reshape_embeddings(bertweet_olid_test_embeddings)

### BERTweet

#### DynaHate

In [8]:
bertweet_dynahate_train_dev = np.concatenate((bertweet_dynahate_train_embeddings, bertweet_dynahate_dev_embeddings))

grid_results = gridsearch.fit(bertweet_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bertweet_dynahate_train_dev)
test_preds = mlp.predict(bertweet_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/bertweet_dynahate")
save_model(mlp, "Saves/bertweet_dynahate")

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'solver': 'adam'}
Accuracy Train Dev:  0.6984928694900605
Accuracy Test:  0.670631067961165
Weighted F1 Train Dev:  0.6903586753156541
Weighted F1 Test:  0.6613119080812634
Macro F1 Train Dev:  0.6855515578094915
Macro F1 Test:  0.6535428289028284
Micro F1 Train Dev:  0.6984928694900605
Micro F1 Test:  0.670631067961165
Weighted Recall Train Dev:  0.6984928694900605
Weighted Recall Test:  0.670631067961165
Macro Recall Train Dev:  0.6871072221531471
Macro Recall Test:  0.6548833236198247
Micro Recall Train Dev:  0.6984928694900605
Micro Recall Test:  0.670631067961165
Confusion Matrix Train Dev: 
[[ 9175  7942]
 [ 3221 16686]]
Confusion Matrix Test: 
[[ 924  928]
 [ 429 1839]]


#### LatentHatred

In [9]:
bertweet_latenthatred_train_dev = np.concatenate((bertweet_latenthatred_train_embeddings, bertweet_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(bertweet_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bertweet_latenthatred_train_dev)
test_preds = mlp.predict(bertweet_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/bertweet_latenthatred")
save_model(mlp, "Saves/bertweet_latenthatred")

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'solver': 'adam'}
Accuracy Train Dev:  0.6957790192427064
Accuracy Test:  0.6884543761638734
Weighted F1 Train Dev:  0.6625899153620184
Weighted F1 Test:  0.652498463257425
Macro F1 Train Dev:  0.4394263541326186
Macro F1 Test:  0.429764674977289
Micro F1 Train Dev:  0.6957790192427064
Micro F1 Test:  0.6884543761638734
Weighted Recall Train Dev:  0.6957790192427064
Weighted Recall Test:  0.6884543761638734
Macro Recall Train Dev:  0.4448717161591799
Macro Recall Test:  0.4344745587600565
Micro Recall Train Dev:  0.6957790192427064
Micro Recall Test:  0.6884543761638734
Confusion Matrix Train Dev: 
[[8845 1098    2]
 [2985 2361    1]
 [ 458  357    3]]
Confusion Matrix Test: 
[[2989  356    1]
 [1046  706    1]
 [ 145  124    2]]


#### OLID

In [12]:
bertweet_olid_train_dev = np.concatenate((bertweet_olid_train_embeddings, bertweet_olid_dev_embeddings))

grid_results = gridsearch.fit(bertweet_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bertweet_olid_train_dev)
test_preds = mlp.predict(bertweet_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/bertweet_olid")

save_model(mlp, "Saves/bertweet_olid")

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'solver': 'adam'}
Accuracy Train Dev:  0.7733383685800604
Accuracy Test:  0.8162790697674419
Weighted F1 Train Dev:  0.7652035851394984
Weighted F1 Test:  0.8003818310148455
Macro F1 Train Dev:  0.7279296548236107
Macro F1 Test:  0.7361163977596693
Micro F1 Train Dev:  0.7733383685800603
Micro F1 Test:  0.8162790697674419
Weighted Recall Train Dev:  0.7733383685800604
Weighted Recall Test:  0.8162790697674419
Macro Recall Train Dev:  0.7169657548334019
Macro Recall Test:  0.7116935483870968
Micro Recall Train Dev:  0.7733383685800604
Micro Recall Test:  0.8162790697674419
Confusion Matrix Train Dev: 
[[7824 1016]
 [1985 2415]]
Confusion Matrix Test: 
[[588  32]
 [126 114]]
